<a href="https://colab.research.google.com/github/vijayshankarrealdeal/GoogleColab/blob/main/FeedForward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')

In [4]:
train_dataset = MNIST(root = '.',train=True,download=True,transform=transforms.ToTensor())
test_dataset = MNIST(root = '.',train=False,download=True,transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [12]:
class Model(nn.Module):
  def __init__(self,input_dim = 784,output_dim = 10,hidden_units = 64):
    super(Model,self).__init__()
    self.res = nn.Sequential(
        nn.Linear(input_dim,hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units,hidden_units*2),
        nn.ReLU(),
        nn.Linear(hidden_units*2,output_dim),
        nn.Softmax(dim = 1)
    )

  def forward(self,x):
    return self.res(x)

In [14]:
loss_fn = nn.CrossEntropyLoss()
model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [15]:
model

Model(
  (res): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)

In [20]:
n_epochs = 10
batch_size = 128

In [19]:
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle=True)
test_loader = DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle=False)

In [23]:
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

In [ ]:
for epoch in range(n_epoch):
  train_loss = []
  for inputs,targets in train_loader:
    inputs,target = inputs.to(device),target.to(device)

    inputs = inputs.view(-1,784)/255.0
    optimizer.zero_grad()

    y_hat = model(inputs)
    loss = loss_fn(y_hat,targets)

    train_loss.append(loss.item())
    loss.backward()
    optimizer.step()
  train_loss  = np.mean(train_loss)

  test_loss = []

  for input,target in test_loader:
    inputs,tar = input.to(device),target.to(device)
    y_test_hat = model(inputs)
    loss_test = loss_fn(y_test_hat,tar)
    test_loss.append(loss_test)
  test_loss = np.mean(test_loss)

  train_losses[epoch] = train_loss
  test_losses[epoch] = test_loss